In [1]:
#Question 1: Performance Comparisons

#You are asked to use three approaches taught in Lab 2 to perform sentiment analysis on the dataset: 1) Using Bing Liu’s Lexicon; 2) Using LM dictionary; 3) Using TextBlob; and 4) Using Vader (either from NLTK or from Vader directly). 

#Please report the following:
#1)	Report Precision, Recall and F measure achieved by each tool. Notice that you will calculate them by comparing your prediction and the gold standard (label 0 and 1). Please present the result in a comparison table and highlight the highest performance. 
#(Hint, you should report precision not accuracy. This means you need to calculate positive precision, negative precision and then average precision)

#2)	Provide your analysis of the performances. If you are in charge of identifying the appropriate software to perform sentiment analysis for movie reviews, which one will you choose? Give 1-2 reasons.

In [4]:
import pandas as pd

In [5]:
# read file
df = pd.read_csv("MovieReview-Sample.csv",header = None, names = ['No', 'Review', 'Score'])
df.head()

,No,Review,Score
0,1,films adapted from comic books have had plenty...,1
1,2,every now and then a movie comes along from a ...,1
2,3,you've got mail works alot better than it dese...,1
3,4,jaws is a rare film that grabs your attent...,1
4,5,moviemaking is a lot like being the general ma...,1


In [6]:
data = df.Review.str.lower()

In [7]:
def count_pos_neg(data, positive_dict, negative_dict):
# count of positive and negative words that appeared in each message
# net count which is calculated by positive count subtracting negative count. 
    poscnt = []
    negcnt = []
    netcnt = []

#running a loop for every row of text data
    for nrow in range(0,len(data)):
        text = data[nrow]
        
        pos_count = 0
        neg_count = 0

#looping every word in positive or negative dictionary
#checking if the word is in the text file - if so the count will go up
        for word in positive_dict :
            if (word in text) :
                pos_count = pos_count + 1

        for word in negative_dict :
            if (word in text) :
                neg_count = neg_count + 1

        net_count = pos_count - neg_count

        poscnt.append(pos_count)
        negcnt.append(neg_count)
        netcnt.append(net_count)

    return (poscnt, negcnt, netcnt)

## 1) Bing Liu's Dictionary

In [8]:
import nltk
nltk.download("opinion_lexicon")

[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\veroi\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


True

In [9]:
#import Bing Liu's dictionary
from nltk.corpus import opinion_lexicon

In [10]:
pos_list_BL=set(opinion_lexicon.positive())
neg_list_BL=set(opinion_lexicon.negative())

In [11]:
df['poscnt_BL'], df['negcnt_BL'], df['compound_BL'] = count_pos_neg(data, pos_list_BL, neg_list_BL)

In [12]:
df[['Review','poscnt_BL','negcnt_BL','compound_BL']].head(5)

,Review,poscnt_BL,negcnt_BL,compound_BL
0,films adapted from comic books have had plenty...,16,36,-20
1,every now and then a movie comes along from a ...,21,33,-12
2,you've got mail works alot better than it dese...,32,28,4
3,jaws is a rare film that grabs your attent...,20,41,-21
4,moviemaking is a lot like being the general ma...,17,24,-7


## 2) LM Dictionary


In [13]:
def read_local_dictionary(file):
    # create dictionary list
    words_dict = []
    with open(file, "r") as f: 
        for line in f:
            t = line.strip().lower()
            words_dict.append(t)
    return words_dict

In [14]:
pos_list_LM = read_local_dictionary('positive-words-LM.txt')
neg_list_LM = read_local_dictionary('negative-words-LM.txt')

In [15]:
df['poscnt_LM'], df['negcnt_LM'], df['compound_LM'] = count_pos_neg(data, pos_list_LM, neg_list_LM)

In [16]:
df[['Review','poscnt_LM','negcnt_LM','compound_LM']].head(5)

,Review,poscnt_LM,negcnt_LM,compound_LM
0,films adapted from comic books have had plenty...,5,11,-6
1,every now and then a movie comes along from a ...,7,12,-5
2,you've got mail works alot better than it dese...,12,7,5
3,jaws is a rare film that grabs your attent...,5,14,-9
4,moviemaking is a lot like being the general ma...,1,6,-5


## 3) TextBlob

In [17]:
from textblob import TextBlob

In [18]:
df["score_TextBlob"] = df["Review"].map(lambda x:TextBlob(x).sentiment.polarity)

In [19]:
df[["Review","score_TextBlob"]].head(5)

,Review,score_TextBlob
0,films adapted from comic books have had plenty...,-0.061036
1,every now and then a movie comes along from a ...,0.088390
2,you've got mail works alot better than it dese...,0.081941
3,jaws is a rare film that grabs your attent...,0.066679
4,moviemaking is a lot like being the general ma...,0.054987


## 4) Vader

In [20]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [21]:
analyzer = SentimentIntensityAnalyzer()
scores = [analyzer.polarity_scores(sentence) for sentence in data]

In [22]:
neg_s = [i["neg"] for i in scores]
neu_s = [i["neu"] for i in scores]
pos_s = [i["pos"] for i in scores]
compound_s = [i["compound"] for i in scores]

In [23]:
df['negscore_Vader'], df['neuscore_Vader'], df['posscore_Vader'], df['compound_Vader'] = neg_s, neu_s, pos_s, compound_s

In [24]:
df[['Review','Score','negscore_Vader','neuscore_Vader','posscore_Vader','compound_Vader']].head(5)

,Review,Score,negscore_Vader,neuscore_Vader,posscore_Vader,compound_Vader
0,films adapted from comic books have had plenty...,1,0.138,0.802,0.060,-0.9905
1,every now and then a movie comes along from a ...,1,0.069,0.833,0.098,0.8319
2,you've got mail works alot better than it dese...,1,0.075,0.765,0.160,0.9887
3,jaws is a rare film that grabs your attent...,1,0.085,0.806,0.109,0.9373
4,moviemaking is a lot like being the general ma...,1,0.037,0.849,0.114,0.9819


In [25]:
df_compound = df[['Review', 'Score', 'compound_BL', 'compound_LM', 'score_TextBlob', 'compound_Vader']]
df_compound

,Review,Score,compound_BL,compound_LM,score_TextBlob,compound_Vader
0,films adapted from comic books have had plenty...,1,-20,-6,-0.061036,-0.9905
1,every now and then a movie comes along from a ...,1,-12,-5,0.088390,0.8319
2,you've got mail works alot better than it dese...,1,4,5,0.081941,0.9887
3,jaws is a rare film that grabs your attent...,1,-21,-9,0.066679,0.9373
4,moviemaking is a lot like being the general ma...,1,-7,-5,0.054987,0.9819
...,...,...,...,...,...,...
1995,"if anything , stigmata should be taken as ...",0,-27,-16,-0.099168,-0.9724
1996,john boorman's zardoz is a goofy cinematic...,0,-8,-4,0.008737,0.9788
1997,the kids in the hall are an acquired taste . i...,0,-2,-8,0.168962,0.9558
1998,there was a time when john carpenter was a gre...,0,-18,-4,0.097989,-0.8765


In [29]:
from sklearn.metrics import precision_score, recall_score, f1_score
from tabulate import tabulate

# Define a function to calculate metrics
def calculate_metrics(df, true_col, pred_col, threshold=0):
    # Convert predicted scores to binary format
    y_pred = [1 if score > threshold else 0 for score in df[pred_col]]
    y_true = df[true_col]
    
    # Calculate precision, recall, and F1-score
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    return precision, recall, f1

# Columns with predicted sentiment scores
pred_columns = ['compound_BL', 'compound_LM', 'score_TextBlob', 'compound_Vader']

# Store the results in a list of lists
results = []
for col in pred_columns:
    precision, recall, f1 = calculate_metrics(df, 'Score', col)
    results.append([col, precision, recall, f1])

# Print the results in a tabular format
headers = ["Method", "Precision", "Recall", "F1-Score"]
print(tabulate(results, headers=headers, tablefmt="grid"))


+----------------+-------------+----------+------------+
| Method         |   Precision |   Recall |   F1-Score |
+================+=============+==========+============+
| compound_BL    |    0.796407 |    0.133 |   0.227935 |
+----------------+-------------+----------+------------+
| compound_LM    |    0.69869  |    0.16  |   0.260374 |
+----------------+-------------+----------+------------+
| score_TextBlob |    0.559199 |    0.921 |   0.695882 |
+----------------+-------------+----------+------------+
| compound_Vader |    0.588898 |    0.732 |   0.652697 |
+----------------+-------------+----------+------------+


In [29]:
#Question 2: Ensemble 
#You are going to using ensemble method to improve the performance of individual tool. 
#Can you think of a way to ensemble the three methods/tools to improve the performance? 
#(Hint 1: you may choose the 3 best performing algorithms to ensemble. 
#There is no need to include inferior algorithms from the previous step. 
#Hint 2: the simplest form or ensemble is a majority vote, or a weighted majority vote based on the algorithm performances). 
#Report your performance improvement (in percentage) over any single models. 


In [37]:
df_ensemble = df_compound.drop(['compound_BL'], axis = 1)
df_ensemble

,Review,Score,compound_LM,score_TextBlob,compound_Vader,ensemble_prediction
0,films adapted from comic books have had plenty...,1,-6,-0.061036,-0.9905,0
1,every now and then a movie comes along from a ...,1,-5,0.088390,0.8319,0
2,you've got mail works alot better than it dese...,1,5,0.081941,0.9887,1
3,jaws is a rare film that grabs your attent...,1,-9,0.066679,0.9373,0
4,moviemaking is a lot like being the general ma...,1,-5,0.054987,0.9819,0
...,...,...,...,...,...,...
1995,"if anything , stigmata should be taken as ...",0,-16,-0.099168,-0.9724,0
1996,john boorman's zardoz is a goofy cinematic...,0,-4,0.008737,0.9788,0
1997,the kids in the hall are an acquired taste . i...,0,-8,0.168962,0.9558,0
1998,there was a time when john carpenter was a gre...,0,-4,0.097989,-0.8765,0


In [39]:
#Calculating Ensemble:

import numpy as np

#Generate ensemble predictions based on a simple majority vote
def majority_vote(row):
    votes = [1 if row['compound_Vader'] > 0 else 0,
             1 if row['compound_LM'] > 0 else 0,
             1 if row['compound_BL'] > 0 else 0]
    return 1 if sum(votes) >= 2 else 0
df_ensemble['ensemble_prediction'] = df.apply(majority_vote, axis=1)
df_ensemble

,Review,Score,compound_LM,score_TextBlob,compound_Vader,ensemble_prediction
0,films adapted from comic books have had plenty...,1,-6,-0.061036,-0.9905,0
1,every now and then a movie comes along from a ...,1,-5,0.088390,0.8319,0
2,you've got mail works alot better than it dese...,1,5,0.081941,0.9887,1
3,jaws is a rare film that grabs your attent...,1,-9,0.066679,0.9373,0
4,moviemaking is a lot like being the general ma...,1,-5,0.054987,0.9819,0
...,...,...,...,...,...,...
1995,"if anything , stigmata should be taken as ...",0,-16,-0.099168,-0.9724,0
1996,john boorman's zardoz is a goofy cinematic...,0,-4,0.008737,0.9788,0
1997,the kids in the hall are an acquired taste . i...,0,-8,0.168962,0.9558,0
1998,there was a time when john carpenter was a gre...,0,-4,0.097989,-0.8765,0


In [40]:
#Calculating Emsemble Precision Recall F-measure:

precision, recall, f1 = calculate_metrics(df_compound, 'Score', 'ensemble_prediction')
print(f"Metrics for emsemble prediction: Precision={precision:.4f}, Recall={recall:.4f}, F1-Score={f1:.4f}")

Metrics for emsemble prediction: Precision=0.7517, Recall=0.2210, F1-Score=0.3416
